# 実践データ科学入門 2020年度木曜4限

# 第4回 その1 Iris 線形回帰やりすぎ

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Irisデータセット読み込み
iris = datasets.load_iris()

# データの特徴量
print(iris.feature_names)

目的変数 Y を PW  
説明変数を SL, SW, PL として線形回帰

In [ ]:
# 線形回帰モデルの準備
reg1 = linear_model.LinearRegression()
reg2 = linear_model.LinearRegression()
reg3 = linear_model.LinearRegression()

# 説明変数と目的変数の設定
Y = iris.data[:, 3]    # 目的変数はPW
X1 = iris.data[:, 0:1] # case1: 説明変数はSLのみ
X2 = iris.data[:, 0:2] # case2: 説明変数はSLとSW
X3 = iris.data[:, 0:3] # case3: 説明変数はSL, SW, PL

# 線形回帰
reg1.fit(X1, Y)
reg2.fit(X2, Y)
reg3.fit(X3, Y)

# 学習した回帰モデルで予測値を出力
Y1p = reg1.predict(X1)
Y2p = reg2.predict(X2)
Y3p = reg3.predict(X3)

# 学習誤差
MSE1 = mean_squared_error(Y, Y1p)
MSE2 = mean_squared_error(Y, Y2p)
MSE3 = mean_squared_error(Y, Y3p)

# MSEを表示
print('case1: MSE = %f'%MSE1)
print('case2: MSE = %f'%MSE2)
print('case3: MSE = %f'%MSE3)

仮に目的変数と説明変数の間に明確な因果関係がなく，偽相関しかなかったとしても  
回帰変数を増やすと MSE は減る

---

実は，説明変数として乱数で生成したデータを回帰変数に増やしても MSE は減ってしまう  
乱数で生成したデータなので，PW とは全く因果関係がないのに減ってしまう

In [ ]:
# 追加する回帰変数の数
cols_max = 200

# MSEを格納するための配列を準備
MSEs = np.zeros(cols_max+3)

# 最初の3つは上ですでに計算したMSE
MSEs[0] = MSE1
MSEs[1] = MSE2
MSEs[2] = MSE3

# 線形回帰モデルの設定
regR = linear_model.LinearRegression()

# 回帰変数用の配列の準備
XX = np.copy(X3)
for cols in range(cols_max):
    # 一様乱数を振る
    R = np.random.rand(Y.size, 1) # (150 x 1)の行列として設定する必要がある
    
    # 説明変数を新しい列として追加する
    XX = np.append(XX, R, axis=1) # 列を追加するのには axis=1
    
    # 学習と予測をまとめてできる
    YRp = regR.fit(XX, Y).predict(XX)
    
    # MSEの値を格納
    MSEs[cols+3] = mean_squared_error(Y, YRp)
    
    # MSEを表示
    print('説明変数の数 %3d: MSE = %5.3e    R^2 = %f'%(cols+4, mean_squared_error(Y, YRp), r2_score(Y, YRp)))
    R2 = 1- np.mean((Y-YRp)**2) / np.var(Y)
    R2a = ( (Y.size-1)*R2 - (cols+3) ) / (Y.size-(cols+3)-1)
    print(R2, R2a)
    print()

In [ ]:
# MSE のプロット
# 横軸は回帰変数の数，縦軸は回帰誤差の平均二乗誤差
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
ax.set_xlabel("number of explanatory variables", size=24)
ax.set_ylabel("MSE", size=24)
ax.tick_params(labelsize=20)
ax.plot(np.arange(1, cols_max+4), MSEs, linewidth=3)

In [ ]:
# MSE の対数プロット
# 横軸は回帰変数の数，縦軸は回帰誤差の平均二乗誤差
fig2 = plt.figure(figsize=(12, 8))
ax2 = plt.axes()
ax2.set_xlabel("number of explanatory variables", size=24)
ax2.set_ylabel("MSE", size=24)
ax2.tick_params(labelsize=20)
ax2.plot(np.arange(1, cols_max+4), MSEs, linewidth=3)
ax2.set_yscale('log')

MSEだけを減らそうと思ったら，全く意味のない変数を足しても構わないのである

実際に学習モデルの出力を確かめてみよう

In [ ]:
fig3 = plt.figure(figsize=(12, 8))
ax3 = plt.axes()
ax3.set_xlabel('SL', size=24)
ax3.set_ylabel('PW', size=24)
ax3.tick_params(labelsize=20)
ax3.scatter(XX[:, 0], Y, s=200, label='true value')
ax3.scatter(XX[:, 0], YRp, s=50, label='prediction')
ax3.legend(fontsize=20)

気持ち悪いくらいに完璧に学習モデルの出力が真値と合っている．

---

### 汎化誤差も見てみよう

学習に用いたデータの出力が真値と合っていることは，学習がきちんと済んだことを示しているだけである．  
統計モデルの確認には学習に用いていないデータに対しても出力が正しく合うかどうかを調べなければならない．

In [ ]:
# MSEを格納するための配列を準備
MSEsTest = np.zeros(cols_max+3)

# 目的変数の設定
Ytrain = np.append(iris.data[0::3, 3], iris.data[1::3, 3])
Ytest  = iris.data[2::3, 3]


### 説明変数：SL ###
Xtrain = np.append(iris.data[0::3, 0:1], iris.data[1::3, 0:1], axis=0)
Xtest  = iris.data[2::3, 0:1]

# 学習でデータでフィットして，テストデータで予測
Ypred = regR.fit(Xtrain, Ytrain).predict(Xtest)

# 汎化誤差
MSEsTest[0] = mean_squared_error(Ytest, Ypred)
print('説明変数の数 %3d: MSE = %f'%(1, MSEs[0]))


### 説明変数：SLとSW ###
Xtrain = np.append(iris.data[0::3, 0:2], iris.data[1::3, 0:2], axis=0)
Xtest  = iris.data[2::3, 0:2]

# 学習でデータでフィットして，テストデータで予測
Ypred = regR.fit(Xtrain, Ytrain).predict(Xtest)

# 汎化誤差
MSEsTest[1] = mean_squared_error(Ytest, Ypred)
print('説明変数の数 %3d: MSE = %f'%(2, MSEs[1]))


### 説明変数：SLとSWとPL ###
Xtrain = np.append(iris.data[0::3, 0:3], iris.data[1::3, 0:3], axis=0)
Xtest  = iris.data[2::3, 0:3]

# 学習でデータでフィットして，テストデータで予測
Ypred = regR.fit(Xtrain, Ytrain).predict(Xtest)

# 汎化誤差
MSEsTest[2] = mean_squared_error(Ytest, Ypred)
print('説明変数の数 %3d: MSE = %f'%(3, MSEs[2]))

In [ ]:
for cols in range(cols_max):
    # 説明変数に一様乱数の列を追加
    R = np.random.rand(Ytrain.size, 1)
    Xtrain = np.append(Xtrain, R, axis=1)
    
    R = np.random.rand(Ytest.size, 1)
    Xtest = np.append(Xtest, R, axis=1)
    
    # 学習でデータでフィットして，テストデータで予測
    Ypred = regR.fit(Xtrain, Ytrain).predict(Xtest)
    
    # 汎化誤差
    MSEsTest[cols+3] = mean_squared_error(Ytest, Ypred)
    print('説明変数の数 %3d: MSE = %f'%(cols+4, MSEs[cols+3]))

In [ ]:
# MSE のプロット
# 横軸は回帰変数の数，縦軸は回帰誤差の平均二乗誤差
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
ax.set_xlabel("number of explanatory variables", size=24)
ax.set_ylabel("MSE", size=24)
ax.tick_params(labelsize=20)
ax.plot(np.arange(1, cols_max+4), MSEs, linewidth=3, label='training error')
ax.plot(np.arange(1, cols_max+4), MSEsTest, linewidth=3, label='test error')

In [ ]:
# MSE の対数プロット
# 横軸は回帰変数の数，縦軸は回帰誤差の平均二乗誤差
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
ax.set_xlabel("number of explanatory variables", size=24)
ax.set_ylabel("MSE", size=24)
ax.tick_params(labelsize=20)
ax.plot(np.arange(1, cols_max+4), MSEs, linewidth=3, label='training error')
ax.plot(np.arange(1, cols_max+4), MSEsTest, linewidth=3, label='test error')
ax.set_yscale('log')
ax.legend(fontsize=20)

In [ ]:
print('学習誤差が最小となる回帰変数の数 = %d'%(np.argmin(MSEs)+4))
print('汎化誤算が最小となる回帰変数の数 = %d'%(np.argmin(MSEsTest)+4))

学習誤差は乱数を付け足してもほぼ 0 まで下げることができる．
一方．汎化誤差は下がらない．

In [ ]:
fig4 = plt.figure(figsize=(12, 8))
ax4 = plt.axes()
ax4.set_xlabel('SL', size=24)
ax4.set_ylabel('PW', size=24)
ax4.tick_params(labelsize=20)
ax4.scatter(Xtest[:, 0], Ytest, s=200, label='true value')
ax4.scatter(Xtest[:, 0], Ypred, s=50, label='prediction')
ax4.legend(fontsize=20)

In [ ]:
reg = linear_model.LinearRegression()
Ypred1 = reg.fit(Xtrain[:, 0:3], Ytrain).predict(Xtest[:, 0:3])

print('%3d変数のモデルの汎化誤差 = %f'%(cols_max+4, mean_squared_error(Ytest, Ypred)))
print('SL, SW, PLを変数とするモデルの汎化誤差 = %f'%mean_squared_error(Ytest, Ypred1))

fig4 = plt.figure(figsize=(12, 8))
ax4 = plt.axes()
ax4.set_xlabel('SL', size=24)
ax4.set_ylabel('PW', size=24)
ax4.tick_params(labelsize=20)
ax4.scatter(Xtest[:, 0], Ytest, s=200, label='true value')
ax4.scatter(Xtest[:, 0], Ypred1, s=50, label='prediction')
ax4.legend(fontsize=20)

汎化誤差は SL, SW, PL の3変数の線形モデルと大して差はない  
学習誤差だけに着目するのは統計モデルを構築する上で適切でないことがわかる

尚，決定係数

$$
R^2 = 1 - \frac{\sum_{i=1}^N (y_i - \hat y_i)^2}{\sum_{i=1}^N (y_i - \bar y)^2} = 1 - \frac{\mathrm{MSE}}{\mathrm{Var}(y)}
$$

は MSE が小さければ 1 に近づく．ここで，$y_i$ は $i$番目の目的変数，$\hat y_i$ は $i$番目のデータの学習モデルによる予測値，$\hat y$, $\mathrm{Var}(y)$ は目的変数 $\{y_i\}_{i=1}^N$ の平均値と標本分散である．

したがって学習データに対してのフィッティングを決定係数を用いて評価するのは危険である．

ここで，平方和の計算の時にデータ数で割らずにそれぞれの平方和の自由度で割る評価もある．

$$
\tilde R^2 = 1 - \frac{\frac1{N-p-1} \sum_{i=1}^N (y_i - \hat y_i)^2}{\frac1{N-1} \sum_{i=1}^N (y_i - \bar y)^2} = 1 - \frac{\frac{N-1}{N-p-1}\mathrm{MSE}}{\tilde{\mathrm{Var}}(y)}
$$

この $\tilde R^2$ を自由度調整済み決定係数と呼ぶ．
$p$ は回帰変数の数，$\tilde{\mathrm{Var}}(y)$ は目的変数 $\{y_i\}_{i=1}^N$ の不偏分散である．

ここで，

$$
\begin{align}
\tilde R^2 & = 1 - \frac{N-1}{N-p-1} \frac{\sum_{i=1}^N (y_i - \hat y_i)^2}{\sum_{i=1}^N (y_i - \bar y)^2} = \frac{N-1}{N-p-1} \left( \frac{N-p-1}{N-1} -  \frac{\sum_{i=1}^N (y_i - \hat y_i)^2}{\sum_{i=1}^N (y_i - \bar y)^2} \right) 
\\
& = \frac{N-1}{N-p-1} \left( 1 -  \frac{\sum_{i=1}^N (y_i - \hat y_i)^2}{\sum_{i=1}^N (y_i - \bar y)^2}\right) - \frac{p}{N-p-1} = \frac{(N-1) R^2 - p}{N-p-1}
\end{align}
$$

となるので，$\tilde R^2 \le 1$ である（負の値も取り得る）．

上の定義による決定係数 $R^2$ は，定数項が含まれる回帰モデルでは非負の値を取るが，原点を通過する回帰モデルでは負の値を取り得ることに注意する．偏差平方和の分解が成立しないためである（各自で確かめてみよう）．

In [ ]:
R2 = np.zeros(cols_max+3)
R2a = np.zeros(cols_max+3)
for cols in range(cols_max+3):
    # 学習でデータでフィットして，テストデータで予測
    Ypred = regR.fit(Xtrain[:, :cols+1], Ytrain).predict(Xtest[:, :cols+1])
    
    # 汎化誤差
    #MSEsTest[cols+3] = mean_squared_error(Ytest, Ypred)
    
    print('説明変数の数 %3d: R^2 = %f'%(cols+4, r2_score(Ytest, Ypred)))    
    R2 = 1- np.mean((Ytest-Ypred)**2) / np.var(Ytest)
    R2a = ( (Ytest.size-1)*R2 - (cols+1) ) / (Ytest.size-(cols+1)-1)
    print(R2, R2a)
    print()



## 演習

scikit-learn に入っている別のデータセット：糖尿病患者の診療データはなかなかフィッティングが難しいデータセットである．  
これに対して，多項式回帰でも重回帰でもよいのでとにかくフィッティングして見せよ．

In [ ]:
# Irisデータセット読み込み
diabetes = datasets.load_diabetes()

# データの特徴量
print(diabetes.feature_names)

- age: 年齢
- sex: 性別
- bmi: BMI = 体重kg / (身長m)^2
- bp: 平均血圧
- s1: 血清測定値1 (tc)
- s2: 血清測定値2 (ldl)
- s3: 血清測定値3 (hdl)
- s4: 血清測定値4 (tch)
- s5: 血清測定値5 (ltg)
- s6: 血清測定値6 (glu)

目標変数は1年後の疾患の進行状況

In [ ]:
#　データセット読み込み
Y = diabetes.target
X = diabetes.data[:, 2:3] # たとえば bmi

plt.scatter(X, Y)

<h3><div style="text-align: right;">以上</div></h3>